# A Simple Neural network
## Part 3: Hidden layer

This tutorial is part 3 of the previous tutorials on neural networks (TODO: url). While the previous tutorials described very simple single layer regression and classification models, this tutorial will describe a 2-class classification neural network with 1 input dimension, and a non-linear hidden layer with 2 dimensions. While we didn't add the bias parameters to the previous 2 models, we will add them to this model. The network of this model is shown in the following figure:

![Image of the logistic model](https://dl.dropboxusercontent.com/u/8938051/Blog_images/SimpleANN03.png)

In [ ]:
# Python imports
import numpy as np # Matrix and vector computation package
import matplotlib.pyplot as plt  # Plotting library
# Allow matplotlib to plot inside this notebook
%matplotlib inline
# Set the seed of the numpy random number generator so that the tutorial is reproducable
np.random.seed(seed=1)

## Define the dataset 

In this example the target classes $t$ will be generated from 2 class distributions: blue ($t=1$) and red ($t=0$). Where the red class is a [multimodal distribution](http://en.wikipedia.org/wiki/Multimodal_distribution) that surrounds the distribution of the blue class. This results in a 1D dataset that is not linearly seperable. The model from part 2 won't be able to classify both classes correctly since it can learn only linear seperators. By adding a hidden layer with a non-linear transfer function the model will be able to train a non-linear classifier.

In [ ]:
# Define and generate the samples
nb_of_samples_per_class = 20  # The number of sample in each class
blue_mean = [0]  # The mean of the blue class
red_left_mean = [-2]  # The mean of the red class
red_right_mean = [2]  # The mean of the red class

std_dev = 0.5  # standard deviation of both classes
# Generate samples from both classes
x_blue = np.random.randn(nb_of_samples_per_class, 1) * std_dev + blue_mean
x_red_left = np.random.randn(nb_of_samples_per_class/2, 1) * std_dev + red_left_mean
x_red_right = np.random.randn(nb_of_samples_per_class/2, 1) * std_dev + red_right_mean


# Merge samples in set of input variables x, and corresponding set of
# output variables t
x = np.vstack((x_blue, x_red_left, x_red_right))
# print x
t = np.vstack((np.ones((x_blue.shape[0],1)), 
               np.zeros((x_red_left.shape[0],1)), 
               np.zeros((x_red_right.shape[0], 1))))

In [ ]:
# Plot samples from both classes as lines on a 1D space
plt.figure(figsize=(8,0.5))
plt.xlim(-4,4)
plt.ylim(-1,1)
# Plot samples
plt.plot(x_blue, np.zeros_like(x_blue), 'b|', ms = 30) 
plt.plot(x_red_left, np.zeros_like(x_red_left), 'r|', ms = 30) 
plt.plot(x_red_right, np.zeros_like(x_red_right), 'r|', ms = 30) 
plt.gca().axes.get_yaxis().set_visible(False)
plt.show()

## Non-linear transfer function 

The non-linear transfer function used in the hidden layer of this example is the [Gaussian](http://en.wikipedia.org/wiki/Gaussian_function) [radial basis function](http://en.wikipedia.org/wiki/Radial_basis_function) (RBF).  
The RBF is a transfer function that is not usally used in neural networks, exept for [radial basis function networks](http://en.wikipedia.org/wiki/Radial_basis_function_network). One of the most common transfer functions in neural networks is the [sigmoid](http://en.wikipedia.org/wiki/Sigmoid_function) transfer function.
The RBF will allow to seperate the blue samples from the red samples in this simple example by only activating for a certain region around the origin. The RBF is plotted in the figure below and is defined in this example as:

$$ \text{RBF} = \phi(z) = e^{-z^2} $$

The derivative of this RBF function is:

$$ \frac{d \phi(z)}{dz} = -2 z e^{-z^2} $$

In [ ]:
# Define the rbf function
def rbf(z): return np.exp(-z**2)

# Plot the rbf function
z = np.linspace(-6,6,100)
plt.plot(z, rbf(z), 'b-')
plt.xlabel('z')
plt.ylabel('$e^{-z^2}$')
plt.title('rbf function')
plt.grid()
plt.show()

## Optimization by backpropagation

We will train this model by using the [backpropagation](http://en.wikipedia.org/wiki/Backpropagation) algorithm that is typically used to train neural networks. Each step in the backpropagation algorithm consists of two steps:

1. A forward propagation step to compute the output of the network.
2. A backward propagation step in which the error at the end of the network is propagated backwards through all the neurons, while updating their parameters.

### 1. Forward step

During the forward step the input will be propagated layer by layer through the network to compute the final output of the network.

The $n$ input samples with $1$ variable each are given as a $n \times 1$ matrix $X = [x_1 \ldots x_n]^T$. These inputs are projected onto the 2 dimension of the hidden layer $H$ by according to: 

$$H = \sigma(X * W_h + b_h) = \frac{1}{1+e^{-(X * W_h + b_h)}} $$

Where $W_h = [w_{h1}, w{h2}]$ the weight matrix, and 

In [ ]:
# Define the logistic function
def logistic(z): return 1 / (1 + np.exp(-z))

# Function to compute the hidden activations
def hidden_activations(x, wh):
    return rbf(x * wh)

# Define output layer feedforward
def output_activations(h , wo):
    return logistic(h * wo)

# Define the neural network function
def nn(x, wh, wo): 
    return output_activations(hidden_activations(x, wh), wo)

# Define the neural network prediction function that only returns
#  1 or 0 depending on the predicted class
def nn_predict(x, wh, wo): 
    return np.around(nn(x, wh, wo))

In [ ]:
# Weights
wh = 1
wo = 1

h = hidden_activations(x, wh)
print 'h.shape: ', h.shape
o = output_activations(h, wo)
print 'o.shape: ', o.shape

y = nn(x, wh, wo)
print 'y.shape: ', y.shape

y_pred = nn_predict(x, wh, wo)
print 'y_pred.shape: ', y_pred.shape

In [ ]:
# Define the cost function
def cost(y, t):
    return - np.sum(np.multiply(t, np.log(y)) + np.multiply((1-t), np.log(1-y)))

In [ ]:
# Plot the cost in function of the weights
# Define a vector of weights for which we want to plot the cost
nb_of_ws = 100 # compute the cost nb_of_ws times in each dimension
wsh = np.linspace(-10, 10, num=nb_of_ws) # weight 1
wso = np.linspace(-10, 10, num=nb_of_ws) # weight 2
ws_x, ws_y = np.meshgrid(wsh, wso) # generate grid
cost_ws = np.zeros((nb_of_ws, nb_of_ws)) # initialize cost matrix
# Fill the cost matrix for each combination of weights
for i in xrange(nb_of_ws):
    for j in xrange(nb_of_ws):
        cost_ws[i,j] = cost(nn(x, ws_x[i,j], ws_y[i,j]) , t)
# Plot the cost function surface
plt.contourf(ws_x, ws_y, cost_ws, 20)
cbar = plt.colorbar()
cbar.ax.set_ylabel('cost')
plt.xlabel('$w_h$')
plt.ylabel('$w_o$')
plt.title('Cost function surface')
plt.grid()
plt.show()

In [ ]:


# Weights and biases
wh = np.asmatrix([[1, 1]])
bh = np.asmatrix([[0, 0]])
wo = np.asmatrix([[1], [1]])
bo = np.asmatrix([[0]])


# Define hidden feedforward
def layer_hidden_ff(x, wh, bh):
    return logistic(x.dot(wh)+bh)

# print layer_hidden_ff(x, wh, bh)

# Define output layer feedforward
def layer_output_ff(h, wo, bo):
    return logistic(h.dot(wo) + bo)

# print layer_output_ff(layer_hidden_ff(x, wh, bh), wo, bo)

# Define the neural network function
def nn(x, wh, bh, wo, bo): 
    return layer_output_ff(layer_hidden_ff(x, wh, bh), wo, bo)

# Define the neural network prediction function that only returns
#  1 or 0 depending on the predicted class
def nn_predict(x, wh, bh, wo, bo): return np.around(nn(x, wh, bh, wo, bo))

# print nn(x, wh, bh, wo, bo)

In [ ]:
# Define the cost function
def cost(y, t):
    return - np.sum(np.multiply(t, np.log(y)) + np.multiply((1-t), np.log(1-y)))

y = nn(x, wh, bh, wo, bo)
print cost(y, t)

h = layer_hidden_ff(x, wh, bh)

# print layer_output_ff(h, wo, bo)
# print t
# print h

# define the gradient function for the output layer
def gradient_output(h, wo, bo, t): 
    return (layer_output_ff(h, wo, bo) - t).T * h

go = gradient_output(h, wo, bo, t)
print go

def gradient_input(x, wh, bh, error_gradient):
    return np.sum(x * error_gradient, axis=0)

print gradient_input(x, wh, bh, go)

# define the update function delta w which returns the 
#  delta w for each weight in a vector
def delta_w(w_k, x, t, learning_rate):
    return learning_rate * gradient(w_k, x, t)